# Assignment 3
All questions are weighted the same in this assignment. This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. All questions are worth the same number of points except question 1 which is worth 17% of the assignment grade.

**Note**: Questions 2-13 rely on your question 1 answer.

In [8]:
import pandas as pd
import numpy as np

# Filter all warnings
import warnings
warnings.filterwarnings('ignore')

### Question 1
Load the energy data from the file `assets/Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](assets/Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **Energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable]`

Convert `Energy Supply` to gigajoules (**Note: there are 1,000,000 gigajoules in a petajoule**). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, e.g. `'Bolivia (Plurinational State of)'` should be `'Bolivia'`.  `'Switzerland17'` should be `'Switzerland'`.

Next, load the GDP data from the file `assets/world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `assets/scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries, and the rows of the DataFrame should be sorted by "Rank".*

In [33]:
data = pd.ExcelFile("EnergyIndicators.xls")
energy = pd.read_excel(data, skiprows=17, skipfooter=38) # numbers are given

energy = energy.drop(energy.columns[1], axis=1) # excluding the columns
energy = energy.drop(energy.columns[0], axis=1)

energy.columns = ["Country", "Energy Supply", "Energy Supply per Capita", "% Renewable"] # renaming the columns

energy = energy.replace({"...": np.nan})

energy["Energy Supply"] = energy["Energy Supply"] * 1000000 # changing the joules

energy["Country"] = energy["Country"].replace({"United States of America": "United States",
                                            "United Kingdom of Great Britain and Nothern Ireland": "United Kingdom",
                                            "China, Hong Kong Special Administrative Region": "Hong Kong"})

energy['Country'] = energy["Country"].replace(r' \(.*\)', '',regex=True) # regex to be replaced
print(energy["Country"].head(30))

0                          Afghanistan
1                              Albania
2                              Algeria
3                       American Samoa
4                              Andorra
5                               Angola
6                             Anguilla
7                  Antigua and Barbuda
8                            Argentina
9                              Armenia
10                               Aruba
11                          Australia1
12                             Austria
13                          Azerbaijan
14                             Bahamas
15                             Bahrain
16                          Bangladesh
17                            Barbados
18                             Belarus
19                             Belgium
20                              Belize
21                               Benin
22                             Bermuda
23                              Bhutan
24                             Bolivia
25    Bonaire, Sint Eusta

In [38]:
GDP = pd.read_csv("world_bank.csv", skiprows=4)

GDP["Country Name"].replace({"Korea, Rep.": "South Korea", "Iran, Islamic Rep.": "Iran", "Hong Kong SAR, China": "Hong Kong"}, inplace=True)
print(GDP)

         Country Name Country Code                            Indicator Name  \
0               Aruba          ABW  GDP at market prices (constant 2010 US$)   
1             Andorra          AND  GDP at market prices (constant 2010 US$)   
2         Afghanistan          AFG  GDP at market prices (constant 2010 US$)   
3              Angola          AGO  GDP at market prices (constant 2010 US$)   
4             Albania          ALB  GDP at market prices (constant 2010 US$)   
..                ...          ...                                       ...   
259       Yemen, Rep.          YEM  GDP at market prices (constant 2010 US$)   
260      South Africa          ZAF  GDP at market prices (constant 2010 US$)   
261  Congo, Dem. Rep.          COD  GDP at market prices (constant 2010 US$)   
262            Zambia          ZMB  GDP at market prices (constant 2010 US$)   
263          Zimbabwe          ZWE  GDP at market prices (constant 2010 US$)   

     Indicator Code          1960      

In [ ]:
ScimEn = pd.read_excel("scimagojr-3.xlsx") # try to run cin

In [ ]:
ScimEn = ScimEn[:15] #pick the initial 15 elements
GDP.rename(columns={"Country Name": "Country"}, inplace=True)
GDP = GDP[['Country Name', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']]

initial_merge = pd.merge(GDP, energy, how="inner", left_on="Country", right_on="Country")
final_merge = pd.merge(initial_merge, ScimEn, how="inner", left_on="Country", right_on="Country")
final_merge.set_index("Country")

### Question 2
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

### Question 3
What are the top 15 countries for average GDP over the last 10 years?

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

### Question 4
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

### Question 5
What is the mean energy supply per capita?

*This function should return a single number.*

### Question 6
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

### Question 7
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

### Question 8

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return the name of the country*

### Question 9
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

### Question 10
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

### Question 11
Use the following dictionary to group the Countries by Continent, then create a DataFrame that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

### Question 12
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a Series with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

### Question 13
Convert the Population Estimate series to a string with thousands separator (using commas). Use all significant digits (do not round the results).

e.g. 12345678.90 -> 12,345,678.90

*This function should return a series `PopEst` whose index is the country name and whose values are the population estimate string*